In [21]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
from supabase import create_client
from dotenv import load_dotenv
import numpy as np

load_dotenv()

True

In [22]:
url = os.environ.get("SUPABASE_URL")
key = os.environ.get("SUPABASE_KEY")
supabase = create_client(url, key)

In [23]:
# Fetch data from the view
response = supabase.table("sector_etf_flow_daily").select("*").execute()
data = response.data
df = pd.DataFrame(data)

# Check columns to identify the date column
date_col = 'date' if 'date' in df.columns else 'asof_date'
if date_col in df.columns:
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(by=date_col)
    print(f"Date column used: {date_col}")
else:
    print("Date column not found.")
    print(df.columns)

Date column used: asof_date


In [24]:
# Calculated X indices for categorical plotting (removes weekends/gaps)
# Note: 'limit' variable ensures y-axis symmetry for flow_pct, assumed to be defined or we set a default
limit = 0.5 

# Unique tickers
tickers = df['ticker'].unique()

# Determine grid size
num_tickers = len(tickers)
cols = 1
rows = (num_tickers + 1) // cols

# Tweak figure size
fig = plt.figure(figsize=(15, 6 * rows)) # Taller to accommodate rows

for i, ticker in enumerate(tickers):
    ax1 = fig.add_subplot(rows, cols, i + 1)
    
    # Filter data for this ticker
    ticker_data = df[df['ticker'] == ticker].copy()

    # Calculate nav_cum_return if not present
    # Assuming 'nav' column exists and is price/value
    if 'nav' in ticker_data.columns:
        ticker_data = ticker_data.sort_values(by=date_col)
        initial_nav = ticker_data['nav'].iloc[0]
        ticker_data['nav_cum_return'] = (ticker_data['nav'] / initial_nav) - 1
    else:
        # Fallback or error handling if nav is missing, though it should be there
        print(f"Warning: 'nav' column missing for {ticker}")
        ticker_data['nav_cum_return'] = 0
    
    # Create categorical x-axis (indices) to hide gaps
    x_indices = np.arange(len(ticker_data))
    
    color_nav = 'tab:blue'
    ax1.set_xlabel('Date')
    ax1.set_ylabel('NAV Cum. Return %', color=color_nav, fontsize=12)
    
    # PLOT using x_indices instead of dates
    ax1.plot(x_indices, ticker_data['nav_cum_return'] * 100, 
             color=color_nav, label='NAV Cum. Return %', linewidth=2)
             
    ax1.tick_params(axis='y', labelcolor=color_nav)
    ax1.yaxis.set_major_formatter(mtick.PercentFormatter())

    ax1.set_title(f'{ticker} - NAV Daily Return % & Flow %', fontsize=16)
    ax1.grid(True, linestyle="--", alpha=0.5)
    
    # --- Custom X-Axis Ticks ---
    # Show approx 10 ticks to avoid crowding
    n_ticks = 10
    if len(ticker_data) > n_ticks:
        step = len(ticker_data) // n_ticks
        tick_locs = x_indices[::step]
        # Format dates as strings
        tick_labels = ticker_data[date_col].dt.strftime('%Y-%m-%d').iloc[::step]
    else:
        tick_locs = x_indices
        tick_labels = ticker_data[date_col].dt.strftime('%Y-%m-%d')
        
    ax1.set_xticks(tick_locs)
    ax1.set_xticklabels(tick_labels, rotation=45, ha='right')

    # --- Flow Plot (Right Axis) ---
    ax2 = ax1.twinx()
    color_flow = 'tab:orange'
    ax2.plot(x_indices, ticker_data['flow_pct'], 
             color=color_flow, marker='x', linestyle='--', label='Flow %', linewidth=2)
    
    ax2.set_ylabel('Flow %', color=color_flow, fontsize=12)
    ax2.tick_params(axis='y', labelcolor=color_flow)
    ax2.yaxis.set_major_formatter(mtick.PercentFormatter(1.0)) 
    
    # Attempt to calculate reasonable limit if available, else default
    # If limit was calculated in previous cells, we should respect it, but redefining here 
    # to be safe or calculating dynamic limit could be better. 
    # For now, using the hardcoded 0.5 or dynamic max if possible.
    # Let's calculate a dynamic limit per chart to be safe.
    # Calculate dynamic limit safely
    # Handle cases where all flow_pct are NaN or empty
    valid_flows = ticker_data['flow_pct'].dropna()
    valid_flows = valid_flows[np.isfinite(valid_flows)]
    if not valid_flows.empty:
        max_val = max(abs(valid_flows.min()), abs(valid_flows.max()))
        limit_val = max(max_val * 1.1, 0.05) # at least 5%
    else:
        limit_val = 0.05
    
    # Check just in case limit_val is still invalid
    if not np.isfinite(limit_val):
        limit_val = 0.05
        
    ax2.set_ylim(-limit_val, limit_val)

    # Add zero line (Solid Red, explicitly at 0)
    ax2.axhline(0, color='red', linestyle='-', linewidth=1.5, alpha=0.8)

plt.tight_layout()
plt.show()
